<a href="https://colab.research.google.com/github/rexbrandy/Neural_Networks/blob/main/text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install 'portalocker>=2.0.0'
exit()

In [1]:
!python -m spacy download en_core_web_sm

2024-01-22 19:59:06.003370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 19:59:06.003429: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 19:59:06.005280: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 19:59:06.016300: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-22 19:59:07.973539: W tensorflow/comp

In [2]:
import torch
from torchtext.datasets import AG_NEWS
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

SEED = 1111

train_iter = iter(AG_NEWS(split=('train')))
tokenizer = get_tokenizer('basic_english')

def yield_tokens(iter):
    for _, text in iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

# The text pipeline converts a text string into a list of integers based on the lookup table defined in the vocabulary.
# The label pipeline converts the label into integers
#
# print(text_pipeline('here is the an example'), label_pipeline('10')) >> [475, 21, 2, 30, 5297], 9
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

[475, 21, 2, 30, 5297] 9


In [3]:
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]

    for _label, _text in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)

        text_list.append(processed_text)
        label_list.append(label_pipeline(_label))
        offsets.append(processed_text.size(0))

    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[: -1]).cumsum(dim=0)
    text_list = torch.cat(text_list)

    return label_list.to(device), text_list.to(device), offsets.to(device)

cuda


In [4]:
from torch import nn

### The Model
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_classes)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)


In [11]:
import time

def train(model, dataloader, optim, criterion, epoch):
    model.train()
    total_accuracy, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optim.zero_grad()

        prediction = model(text, offsets)

        loss = criterion(prediction, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optim.step()

        total_accuracy += (prediction.argmax(1) == label).sum().item()
        total_count += label.size(0)

        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(f'Epoch: {epoch} | Total {idx}/{len(dataloader)} | Accuracy {total_accuracy / total_count}')

            total_accuracy, total_count = 0, 0
            start_time = time.time()


def evaluate(model, dataloader, criterion):
    model.eval()
    correct = {classname: 0 for classname in classes}
    total = {classname: 0 for classname in classes}
    total_accuracy, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offset) in enumerate(dataloader):
            prediction = model(text, offset)
            loss = criterion(prediction, label)



            total_accuracy += (prediction.argmax(1) == label).sum().item()
            total_count += label.size(0)

    return total_accuracy / total_count

In [7]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

n_epochs = 10
lr = 5
batch_size = 64

train_iter, test_iter = AG_NEWS()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch
)


embed_size = 64
classes = set([label for (label, text) in train_iter])
num_classes = len(classes)
vocab_len = len(vocab)

model = TextClassificationModel(vocab_len, embed_size, num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

for epoch in range(1, n_epochs+1):
    epoch_start_time = time.time()
    train(model, train_dataloader, optimizer, criterion, epoch)
    accu_val = evaluate(model, valid_dataloader, criterion)

    if total_accu is not None and total_accu > accu_val:
        scheduler.step()

    else:
        total_accu = accu_val

    print('-'*59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print('-'*59)

Epoch: 1 | Total 500/1782 | Accuracy 0.6816367265469062
Epoch: 1 | Total 1000/1782 | Accuracy 0.85428125
Epoch: 1 | Total 1500/1782 | Accuracy 0.87578125
-----------------------------------------------------------
| end of epoch   1 | time:  9.52s | valid accuracy    0.261 
-----------------------------------------------------------
Epoch: 2 | Total 500/1782 | Accuracy 0.8976422155688623
Epoch: 2 | Total 1000/1782 | Accuracy 0.8995625
Epoch: 2 | Total 1500/1782 | Accuracy 0.90484375
-----------------------------------------------------------
| end of epoch   2 | time:  9.35s | valid accuracy    0.253 
-----------------------------------------------------------
Epoch: 3 | Total 500/1782 | Accuracy 0.9210953093812375
Epoch: 3 | Total 1000/1782 | Accuracy 0.91896875
Epoch: 3 | Total 1500/1782 | Accuracy 0.92071875
-----------------------------------------------------------
| end of epoch   3 | time:  9.59s | valid accuracy    0.258 
--------------------------------------------------------

In [12]:
print("Checking the results of test dataset.")
accu_test = evaluate(model, test_dataloader, criterion)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy    0.903
